## Imports ##

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/1-raw/openpowerlifting-2025-09-27-31932eca.csv")

print(df.columns)
print(len(df.index))

/var/folders/wm/mz0f2pm95tn34tffq_n21nv80000gn/T/ipykernel_35537/1413922412.py:1: DtypeWarning: Columns (31,33,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/1-raw/openpowerlifting-2025-09-27-31932eca.csv")


Index(['Name', 'Sex', 'Event', 'Equipment', 'Age', 'AgeClass',
       'BirthYearClass', 'Division', 'BodyweightKg', 'WeightClassKg',
       'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Best3SquatKg',
       'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Best3BenchKg',
       'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg',
       'Best3DeadliftKg', 'TotalKg', 'Place', 'Dots', 'Wilks', 'Glossbrenner',
       'Goodlift', 'Tested', 'Country', 'State', 'Federation',
       'ParentFederation', 'Date', 'MeetCountry', 'MeetState', 'MeetTown',
       'MeetName', 'Sanctioned'],
      dtype='object')
3673663


In [13]:
# Checking unique divisions to understand the data to only keep Open
print("Unique divisions in IPF Raw SBD:")
print(df[
    (df['ParentFederation'] != 'IPF') & 
    (df['Equipment'] == 'Raw') & 
    (df['Event'] == 'SBD')]
    ['Division'].value_counts().head(15))
    # ['AgeClass'].value_counts().head(15))
# print(df['ParentFederation'].value_counts())

Unique divisions in IPF Raw SBD:
Division
Open                142276
MR-O                 31006
Juniors 20-23        23206
FR-O                 18372
MR-Jr                13988
Amateur Open         10390
Submasters 35-39      9801
Masters 40-44         8593
Juniors 18-19         7119
MR-T3                 6964
MR-C                  6168
Pro Open              5876
Masters 45-49         5691
FR-Jr                 5151
Juniors 16-17         5079
Name: count, dtype: int64


In [ ]:
# Event: only SBD
# Equipment: Raw only
# Age: drop empty fields
# Division: Open only (catches all with '-O', excludes lowercase 'o' such as 'Juniors' or 'Over 40')
# TotalKg: drop empty fields (no DQs)
# Place: must be a number (no DQs)
# ParentFederation: only IPF
# Best3: ensure all 3 lifts are successful (no DQs)

data = df[
    (df['Event'] == 'SBD') & 
    (df['Equipment'] == 'Raw') &
    (df['Age'].notna()) &
    # (df['Division'].str.contains(r'O|-O|Open', na=False)) &
    (df['Division'].isin(['Open', 'MR-O', 'FR-O'])) &
    (df['TotalKg'].notna()) &
    (df['Place'].str.isnumeric()) &
    (df['ParentFederation'] == 'IPF') &
    (df['Best3SquatKg'].notna()) &
    (df['Best3BenchKg'].notna()) &
    (df['Best3DeadliftKg'].notna()) 
].copy()

essential_columns = [
    'Name', 'Date', 'Sex', 'Age', 'BodyweightKg',
    'Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg', 
    'TotalKg', 'Dots'
]

data = data[essential_columns].copy()

# Convert to datetime
data['Date'] = pd.to_datetime(data['Date'])
# Sort by lifter and date (important for feature engineering)
data = data.sort_values(['Name', 'Date']).reset_index(drop=True)

data.to_csv('../data/2-preprocessed/cleanIPF_minimal.csv', index=False)

print(f"Original rows: {len(df)}")
print(f"Filtered rows: {len(data)}")
print(f"Unique lifters: {data['Name'].nunique()}")
print(f"Male: {len(data[data['Sex']=='M'])}, Female: {len(data[data['Sex']=='F'])}")



Original rows: 3673663
Filtered rows: 194769
Unique lifters: 90879
Male: 126969, Female: 67800
